In [2]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv

from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [ ]:
mbti = """
ISTJ
The Logistician
These individuals tend to be serious, matter-of-fact, and reserved. They appreciate order and organization and pay a great deal of attention to detail.
They like to plan things out in advance and place an emphasis on tradition and law. They are responsible and realistic and can be described as dependable and trustworthy.


ISFJ
The Defender
These individuals are friendly, responsible, and reserved. They are service and work-oriented, committing to meeting their obligations and duties.
They are loyal, considerate, and place a lot of focus on the care of others. They are non-confrontational and value an orderly and harmonious environment.


INFJ
The Advocate
People with this personality type are serious, logical and hardworking. They are also compassionate, conscientious, and reserved.
They value close, deep connections and are sensitive to the needs of others, but also need time and space alone to recharge.


INTJ  The Architect
These people are highly independent, self-confident and prefer to work alone. They are analytical, creative, logical, and driven.
They place an emphasis on logic and fact rather than emotion and can be viewed as perfectionist.
They tend to have high expectations of competence and performance for themselves and others.


ISTP
The Crafter
People with this personality type are fearless and independent. They love adventure, new experiences, and risk-taking.
They tend to be quiet observers and are not well attuned to the emotional states of others, sometimes coming across as insensitive or stoic.
They are results- oriented, acting quickly to find workable solutions and understand the underlying cause of practical problems.


ISFP
The Artist
These individuals are quiet, friendly, easy going, and sensitive. They have a strong need for personal space and time alone to recharge.
They value deep connection and prefer to spend time with smaller groups of close friends and family.
They are highly considerate and accepting, avoiding confrontation and committed to their values and to people who are important to them.


INFP
The Mediator
These people are creative, idealistic, caring, and loyal. They have high values and morals, and are constantly seeking out ways to understand people and to best serve humanity.
They are family and home-oriented and prefer to interact with a select group of close friends.


INTP
The Thinker
People with this personality type are described as quiet, contained, and analytical. They are highly focused on how things work and on solving problems, and tend to be good at logic and math.
They are more interested in ideas and theoretical concepts than in social interaction. They are loyal and affectionate to their closest friends and family, but tend to be difficult to get to know.


ESTP
The Entrepreneur
These individuals are action-oriented, taking pragmatic approaches to obtain results and solve problems quickly. They are often sophisticated, charming, and spontaneous.
They are outgoing and energetic, and enjoy spending time with a wide circle of friends and acquaintances. They focus on the here and now and prefer the practical over the abstract.


ESFP
The Entertainer
These people tend to be outgoing, friendly, and impulsive, seizing energy from other people. They love to be the center of attention and enjoy working with others in new environments.
They can be described as easy going, fun, and optimistic. They are spontaneous and focused on the present moment, and enjoy learning through hands-on experiences with other people.


ENFP
The Champion
These individuals are enthusiastic, creative, energetic, and highly imaginative. They have excellent people and communication skills and are good at giving others appreciation and support.
They do, however, seek approval from others. They value emotions and expression. They dislike routine and might struggle with disorganization and procrastination.


ENTP
The Debater
People with this personality type can be described as innovative, outspoken, and lively. They are idea-oriented and are more focused on the future rather than on the present moment.
They enjoy interacting with a wide variety of people and love to engage with others in debates. They tend to be easy to get along with, but also can be argumentative at times. They are great conversationalists and make good entrepreneurs.


ESTJ
The Director
These people are responsible, practical, and organized. They are assertive and like to take charge, focused on getting results in the most efficient way possible. They have clear standards and place a high value on tradition and rules.
They can be seen as rigid, stubborn, or bossy as they are forceful in implementing their plans. However, they tend to excel at putting plans into action because they are hardworking, self-confident, and dependable.


ESFJ
The Caregiver
These individuals are warmhearted, conscientious, and harmonious. They wear their hearts on their sleeves and tend to see the best in others.
They enjoy helping others and providing the care that people need, but want to be appreciated and noticed for their contributions. They are careful observers of others and excel in situations involving personal contact and community.


ENFJ
Protagonist
These people are responsible, warm, and loyal. They are highly attuned to the emotions of others and capable of forging friendships with essentially anybody.
They have a desire to help others fulfill their potential, and they derive personal satisfaction from helping others. They tend to make good leaders as they are highly capable of facilitating agreement among diverse groups of people.


ENTJ
The Commander
These individuals like to take charge. They value organization and structure and appreciate long-term planning and goal setting.
They have strong people skills and enjoy interacting with others, but they are not necessarily attuned to their own emotions or the emotions of others.
They have strong leadership skills and tend to make good executives, captains, and administrators.
"""

In [15]:

import random
from datasets import load_dataset, Dataset
import json
from openai.types.chat.chat_completion import ChatCompletion
import os


process_details = load_dataset("lucaelin/generic_process_details_v1", split="train")

personalized_response_schema = {
    "type": "object",
    "properties": {
        "responses_examples": {
            "type": "array",
            "items": {
                "type": "string",
                "description": "The response this personality would give."
            },
            "minItems": 4,
        },
    },
    "required": ["response"],
    "additionalProperties": False
}

def generate_situation_description(entry):
    results = []

    available_models = [
        "mistralai/mistral-nemo",
        #"mistralai/mixtral-8x7b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-small",
        #"mistralai/mistral-medium",
        "mistralai/mistral-large"
    ]

    model = random.choice(available_models)

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert ..."
        }, {
            "role": "user",
            "content": json.dumps(entry)+'\nUse the generate_personality_response function to generate a personalized response to the input.'
        }],
        tools=[{
            "type": "function",
            "function": {
                "name": "generate_personality_response",
                "parameters": personalized_response_schema,
            },
        }],
        tool_choice="required",
        max_tokens=8191,
        temperature=1.2,
        extra_body={
            "provider": {
                "require_parameters": True,
            },
        }
    )

    personalized_response = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)


    results.append({
        "profession": entry["profession"], # TODO
        "process": entry["process"], # TODO
        "responses": personalized_response["responses_examples"],
    })
    
    return results


domain_situations = process_details.shuffle().map(
    sane_batch_map(generate_situation_description, {"profession": [], "process": [], "responses": []}),
    batched=True, 
    batch_size=4, 
    num_proc=4, 
    remove_columns=process_details.column_names
)
#domain_actions.sort(["profession", "process"]).to_list()
domain_situations.sort(["profession", "process"]).push_to_hub("lucaelin/personalized_responses_v1")

Filter: 100%|██████████| 821/821 [00:00<00:00, 37577.46 examples/s]mples]


{'profession': 'School Psychologist', 'process': 'Parent Meetings', 'description': "In the process of Parent Meetings, school psychologists collaborate with parents and teachers to discuss a student's progress, share insights, address concerns, and establish a plan for future success. This typically involves reviewing assessment results, academic performance, and behavioral observations. The goal is to provide a comprehensive understanding of the student's strengths and challenges, and to develop a tailored plan that supports their educational and emotional well-being.", 'location1': 'indoor', 'location2': 'office', 'location_name': 'school', 'challenge': 'mental', 'duration': '30m', 'frequency': 'as needed', 'importance': 'null', 'equipment': '[{"name": "standardized assessment tools", "description": "Use tools such as cognitive, academic, and behavioral assessments to evaluate the student\'s abilities and needs."}, {"name": "documentation", "description": "Maintain accurate records o

Filter: 100%|██████████| 821/821 [00:00<00:00, 29643.89 examples/s]


{'profession': 'Janitor', 'process': 'Supply Restocking', 'description': "Supply restocking involves replenishing cleaning and maintenance supplies in various locations across a facility to ensure continuous availability for janitorial tasks. It's typically done on a daily or weekly basis, involving careful inventory management and distribution of supplies. The process promotes efficient and effective cleaning and maintenance operations, ensuring that all necessary supplies are readily available when needed, and helps maintain the overall cleanliness and hygiene of the facility.", 'location1': 'indoor', 'location2': 'office', 'location_name': 'Storage Room', 'challenge': 'physical', 'duration': '1h', 'frequency': 'daily', 'importance': 'null', 'equipment': '[{"name": "dollies", "description": "Used for moving heavy supplies such as drums of chemicals or large boxes of paper products."}]', 'localities': '[{"name": "Storage Room", "description": "Primary location where supplies are store

Filter: 100%|██████████| 821/821 [00:00<00:00, 42442.95 examples/s]mples]


...

{'profession': 'Financial Manager', 'process': 'Budget Planning', 'description': 'Budget planning is the process of creating a plan to spend the money of a business, organization, or individual. It involves estimating the revenue and expenses for an upcoming period and comparing it to the actual results to identify any variances and make necessary adjustments. The outcome should be a detailed budget that provides a clear picture of the financial health of the organization and enables informed decision-making.', 'location1': 'indoor', 'location2': 'office', 'location_name': 'Financial Department', 'challenge': 'mental', 'duration': '1d', 'frequency': 'monthly', 'importance': 'null', 'equipment': '[{"name": "laptop", "description": "A laptop is used to create and maintain budget spreadsheets."}, {"name": "calculator", "description": "A calculator is used to perform calculations when creating the budget."}]', 'localities': '[{"name": "Financial Manager\'s office", "description": "The budg

Filter: 100%|██████████| 821/821 [00:00<00:00, 22252.17 examples/s]


{'profession': 'Computer Systems Administrator', 'process': 'Performance Monitoring', 'description': "Computer Systems Administrators are responsible for monitoring and managing the performance of computer systems to ensure they are running optimally and efficiently. During performance monitoring, the administrator monitors system metrics, such as CPU usage, memory usage, network traffic, and disk input/output. They use various tools, such as Windows Performance Monitor or other third-party tools, to gather information about the system's performance. They analyze the data collected and identify any bottlenecks or issues that may be impacting the system's performance. The administrator may also perform tasks such as configuring system parameters, installing patches or updates, and troubleshooting issues to improve system performance. The ultimate goal of performance monitoring is to ensure that the system is running smoothly and efficiently, minimizing downtime and improving overall sys

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.69s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/domain_situations_v1/commit/00f48250a677b99c969c407abfa2ece0a0a4d62a', commit_message='Upload dataset', commit_description='', oid='00f48250a677b99c969c407abfa2ece0a0a4d62a', pr_url=None, pr_revision=None, pr_num=None)